In [1]:
from math import pi
import numpy as np
from numpy import log, exp
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import Column
from scipy.constants import c
from scipy.interpolate import interp1d

output_notebook()

Loading BokehJS ...

In [28]:
delta_T = 1e-9
t = np.arange(start=0, stop=10e-6, step=delta_T)
Q0 = 0.8e5
beta = 5
omega = 2*pi * 3e9
Tc = (2*Q0) / (omega * (1+beta))
alpha = (2*beta) / (1+beta)

def calc_Ek(t):
    t1 = 4e-6
    t2 = t1 + 0.1e-6
    m = -1e6
    if t<t1:
        return -1.0
    elif t<t2:
        return 1.0
    else:
        val = 1.0 + m*(t-t2)
        if val<0:
            val = 0.0
        return val

Ek = np.array([calc_Ek(i) for i in t])

Ee = [0.0]
for val in Ek:
    Ee.append((1/(Tc+delta_T)) * (Tc*Ee[-1] - delta_T*alpha*val))
Ee = np.array(Ee[1:])

EL = Ee + Ek

In [29]:
fig1 = figure(width=900, height=200)
fig2 = figure(width=900, height=200)
fig3 = figure(width=900, height=200)

fig1.line(t, Ek, color='red')
fig2.line(t, Ee, color='black')
fig3.line(t, EL, color='blue')

show(Column(fig1, fig2, fig3))

In [30]:
EL_func = interp1d(x=t, y=EL, kind='linear', bounds_error=False, fill_value=0.0, assume_sorted=True)

def E_cavity(z_prime, t):
    L = 3.0
    T_a = (L / (g*vg)) * log(1 / (1 - g))
    
    delta_t = T_a * (log(1 - g*z_prime) / log(1 - g))
    
    return EL_func(t - delta_t)

def acc_voltage(t):
    z = np.arange(0, 1, 0.001)
    E = E_cavity(z, t)
    return sum(E) * (z[1] - z[0])

fig = figure(width=900, height=400)

g = 0.01/3

vg = 0.03 * c
V = [acc_voltage(t_step) for t_step in t]
fig.line(t, V, color='red')

show(fig)

In [5]:
figs = []
for time_step in np.arange(0, 10e-6, 0.5e-6):
    figs.append(figure(width=900, height=200))
    z = np.arange(0, 1, 0.001)
    figs[-1].line(z, E_cavity(z, time_step))

show(Column(*figs))